In [1]:
import tensorflow as tf
tf.random.set_seed(10)
import numpy as np
np.random.seed(10)
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
# number of turbines
num_turbines = 25

# import data
configs = os.listdir('./SCADA_sectors_v3')

In [3]:
def scada_data_extractor(fname):
    raw_data = np.loadtxt(fname,delimiter=',',skiprows=1)
    
    # Time
    time_data = raw_data[:,0:1]
    
    # Wind data - this will be added in later
    wd = raw_data[:,1:2] 
    
    # wind speeds at each turbine
    windspeed_data = raw_data[:,2:2+num_turbines].reshape(-1,25,1)
    
    # TI at each turbine
    ti_data = raw_data[:,2+num_turbines:2+2*num_turbines].reshape(-1,25,1)
    
    # power data at each turbine
    p_data = raw_data[:,2+2*num_turbines:2+3*num_turbines].reshape(-1,25,1)
    
    # x data at each turbine
    x_data = raw_data[:,2+3*num_turbines:2+5*num_turbines:2].reshape(-1,25,1)
    
    # y data at each turbine
    y_data = raw_data[:,2+3*num_turbines+1:2+5*num_turbines+1:2].reshape(-1,25,1)
    
    # copy over wd data 
    wd_data = np.zeros(shape=(raw_data.shape[0],25,1))
    wd_data[:,:,0] = wd[:,None,0]
    
    feature_data_ip = np.concatenate((wd_data,windspeed_data,ti_data,x_data,y_data),axis=-1)
    
    return feature_data_ip, p_data

In [4]:
amat_list= []
pos_list = []
ip_list = []
op_list = []

for config in configs:
    fname = './SCADA_sectors_v3/'+config+'/SCADA.csv'
    feature_data_ip, feature_data_op = scada_data_extractor(fname)
    
    ip_list.append(feature_data_ip)
    op_list.append(feature_data_op)
           
    amat = np.loadtxt('./SCADA_sectors_v3/'+config+'/node_interactions.csv',delimiter=',')
    pos = np.loadtxt('./SCADA_sectors_v3/'+config+'/positions.csv',delimiter=',',skiprows=1)
    
    for i in range(feature_data_ip.shape[0]):
        amat_list.append(amat.copy())
        pos_list.append(pos.copy())

In [5]:
ip_data = ip_list[0]
op_data = op_list[0]

for i in range(1,len(ip_list)):
    temp = ip_list[i]
    ip_data = np.concatenate((ip_data,temp),axis=0)
    
    temp = op_list[i]
    op_data = np.concatenate((op_data,temp),axis=0)

amat_data = np.asarray(amat_list)
pos_data = np.asarray(pos_list) # Ignoring pos_data for now but that can be add to ip_data

In [6]:
amat_data = amat_data.reshape(-1,amat_data.shape[1]*amat_data.shape[2],1)
print(amat_data.shape)

(10181, 625, 1)


In [7]:
ip_data = ip_data.astype('float32')
amat_data = amat_data.astype('float32')
op_data = op_data[:,:,0]

## Build Graph MPNN

In [8]:
class Message_Passer(tf.keras.layers.Layer):
    def __init__(self, node_dim):
        super(Message_Passer, self).__init__()
        self.node_dim = node_dim
        self.nn = tf.keras.layers.Dense(units=self.node_dim*self.node_dim, activation = tf.nn.relu)
      
    def call(self, node_j, edge_ij):
        
        # Embed the edge as a matrix
        A = self.nn(edge_ij)
        
        # Reshape so matrix mult can be done
        A = tf.reshape(A, [-1, self.node_dim, self.node_dim])
        node_j = tf.reshape(node_j, [-1, self.node_dim, 1])
        
        # Multiply edge matrix by node and shape into message list
        messages = tf.linalg.matmul(A, node_j)
        messages = tf.reshape(messages, [-1, tf.shape(edge_ij)[1], self.node_dim])

        return messages
    
class Message_Agg(tf.keras.layers.Layer):
    def __init__(self):
        super(Message_Agg, self).__init__()
    
    def call(self, messages):
        return tf.math.reduce_sum(messages, 2)
    

class Update_Func_GRU(tf.keras.layers.Layer):
    def __init__(self, state_dim):
        super(Update_Func_GRU, self).__init__()
        self.concat_layer = tf.keras.layers.Concatenate(axis=1)
        self.GRU = tf.keras.layers.GRU(state_dim)
        
    def call(self, old_state, agg_messages):
    
        # Remember node dim
        n_nodes  = tf.shape(old_state)[1]
        node_dim = tf.shape(old_state)[2]
        
        # Reshape so GRU can be applied, concat so old_state and messages are in sequence
        old_state = tf.reshape(old_state, [-1, 1, tf.shape(old_state)[-1]])
        agg_messages = tf.reshape(agg_messages, [-1, 1, tf.shape(agg_messages)[-1]])
        concat = self.concat_layer([old_state, agg_messages])
        
        # Apply GRU and then reshape so it can be returned
        activation = self.GRU(concat)
        activation = tf.reshape(activation, [-1, n_nodes, node_dim])
        
        return activation
    
# Define the final output layer 
class Output_Regressor(tf.keras.layers.Layer):
    def __init__(self, n_nodes, intermediate_dim):
        super(Output_Regressor, self).__init__()
        self.concat_layer = tf.keras.layers.Concatenate()
        self.hidden_layer_1 = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
        self.hidden_layer_2 = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
        self.hidden_layer_3 = tf.keras.layers.Dense(units=1, activation=tf.nn.relu)
        self.output_layer = tf.keras.layers.Dense(units=n_nodes, activation=None)

        
    def call(self, nodes, edges):
                   
        # Remember node dims
        n_nodes  = tf.shape(nodes)[1]
        node_dim = tf.shape(nodes)[2]
        
        # Tile and reshape to match edges
        state_i = tf.reshape(tf.tile(nodes, [1, 1, n_nodes]),[-1,n_nodes*n_nodes, node_dim ])
        state_j = tf.tile(nodes, [1, n_nodes, 1])
        
        # concat edges and nodes and apply MLP
        concat = self.concat_layer([state_i, edges, state_j])
                
        activation_1 = self.hidden_layer_1(concat)  
        activation_2 = self.hidden_layer_2(activation_1)
        activation_3 = self.hidden_layer_3(activation_2)
        
        output_value = self.output_layer(activation_3[:,:,0])
        
        return output_value

In [9]:
# Define a single message passing layer
class MP_Layer(tf.keras.layers.Layer):
    def __init__(self, state_dim):
        super(MP_Layer, self).__init__(self)
        self.message_passers  = Message_Passer(node_dim = state_dim) 
        self.message_aggs    = Message_Agg()
        self.update_functions = Update_Func_GRU(state_dim = state_dim)
        
        self.state_dim = state_dim         

    def call(self, nodes, edges, mask):
      
        n_nodes  = tf.shape(nodes)[1]
        node_dim = tf.shape(nodes)[2]
        
        state_j = tf.tile(nodes, [1, n_nodes, 1])

        messages  = self.message_passers(state_j, edges)

        # Do this to ignore messages from non-existant nodes
        masked =  tf.math.multiply(messages, mask)
        
        masked = tf.reshape(masked, [tf.shape(messages)[0], n_nodes, n_nodes, node_dim])

        agg_m = self.message_aggs(masked)
        
        updated_nodes = self.update_functions(nodes, agg_m)
        
        nodes_out = updated_nodes
        # Batch norm seems not to work. 
        #nodes_out = self.batch_norm(updated_nodes)
        
        return nodes_out

In [10]:
adj_input = tf.keras.Input(shape=(None,), name='adj_input')
nod_input = tf.keras.Input(shape=(None,), name='nod_input')
class MPNN(tf.keras.Model):
    def __init__(self, n_nodes, out_int_dim, state_dim, T):
        super(MPNN, self).__init__(self)   
        self.T = T
        self.embed = tf.keras.layers.Dense(units=state_dim, activation=tf.nn.relu)
        self.MP = MP_Layer(state_dim)     
        self.edge_regressor  = Output_Regressor(n_nodes,out_int_dim)
        #self.batch_norm = tf.keras.layers.BatchNormalization() 

        
    def call(self, inputs):
      
        nodes = inputs[0]
        edges = inputs[1]

        # Get distances, and create mask wherever 0 (i.e. non-existant nodes)
        # This also masks node self-interactions...
        # This assumes distance is last
        len_edges = tf.shape(edges)[-1]
        
        _, x = tf.split(edges, [len_edges -1, 1], 2)
        mask =  tf.where(tf.equal(x, 0), x, tf.ones_like(x))
        
        # Embed node to be of the chosen node dimension (you can also just pad)
        nodes = self.embed(nodes) 
        
        #nodes = self.batch_norm(nodes)
        # Run the T message passing steps
        for mp in range(self.T):
            nodes =  self.MP(nodes, edges, mask)
        
        # Regress the output values
        con_edges = self.edge_regressor(nodes, edges)
        
        
        return con_edges

In [11]:
def mse(orig , preds):
 
    # Mask values for which no scalar coupling exists
    mask  = tf.where(tf.equal(orig, 0), orig, tf.ones_like(orig))

    nums  = tf.boolean_mask(orig,  mask)
    preds = tf.boolean_mask(preds,  mask)

    reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(nums, preds)))

    return reconstruction_error

def log_mse(orig , preds):
 
    # Mask values for which no scalar coupling exists
    mask  = tf.where(tf.equal(orig, 0), orig, tf.ones_like(orig))

    nums  = tf.boolean_mask(orig,  mask)
    preds = tf.boolean_mask(preds,  mask)


    reconstruction_error = tf.math.log(tf.reduce_mean(tf.square(tf.subtract(nums, preds))))


    return reconstruction_error

def mae(orig , preds):
 
    # Mask values for which no scalar coupling exists
    mask  = tf.where(tf.equal(orig, 0), orig, tf.ones_like(orig))

    nums  = tf.boolean_mask(orig,  mask)
    preds = tf.boolean_mask(preds,  mask)


    reconstruction_error = tf.reduce_mean(tf.abs(tf.subtract(nums, preds)))


    return reconstruction_error

def log_mae(orig , preds):
 
    # Mask values for which no scalar coupling exists
    mask  = tf.where(tf.equal(orig, 0), orig, tf.ones_like(orig))

    nums  = tf.boolean_mask(orig,  mask)
    preds = tf.boolean_mask(preds,  mask)

    reconstruction_error = tf.math.log(tf.reduce_mean(tf.abs(tf.subtract(nums, preds))))

    return reconstruction_error

In [12]:
learning_rate = 0.001
def step_decay(epoch):
    initial_lrate = learning_rate
    drop = 0.1
    epochs_drop = 20.0
    lrate = initial_lrate * np.power(drop,  
           np.floor((epoch)/epochs_drop))
    tf.print("Learning rate: ", lrate)
    return lrate

lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 15, restore_best_weights=True)

#lrate  =  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
#                              patience=5, min_lr=0.00001, verbose = 1)

opt = tf.optimizers.Adam(learning_rate=learning_rate)

In [13]:
mpnn = MPNN(n_nodes=25, out_int_dim = 32, state_dim = 32, T = 3)
mpnn.compile(opt, log_mae, metrics = [mae, log_mse])

## Call network once to build weights

In [14]:
mpnn.call([ip_data[:10],amat_data[:10]])

<tf.Tensor: shape=(10, 25), dtype=float32, numpy=
array([[-0.00675847, -0.01622743,  0.0053461 , -0.00250022,  0.00033265,
        -0.00907387, -0.01168793, -0.00800502, -0.00473904, -0.0045182 ,
        -0.00100265,  0.0040614 ,  0.00327675, -0.00491253, -0.01268168,
         0.01066714, -0.02237931,  0.00214788,  0.01410059,  0.00194093,
        -0.00308206,  0.00755948,  0.01235848, -0.00216651, -0.00664559],
       [-0.00671818, -0.01627544,  0.00525346, -0.00254929,  0.00039963,
        -0.00917484, -0.01164242, -0.00802445, -0.00472527, -0.00452508,
        -0.00101762,  0.0040253 ,  0.00330114, -0.00492937, -0.01258837,
         0.01071505, -0.02236227,  0.00208531,  0.01416406,  0.001931  ,
        -0.00306422,  0.00751599,  0.01241984, -0.00219275, -0.00665756],
       [-0.0068213 , -0.01632321,  0.00541178, -0.00250969,  0.00031634,
        -0.00910692, -0.01176503, -0.00802546, -0.00477567, -0.00449708,
        -0.00100843,  0.0040578 ,  0.00322756, -0.00494221, -0.01280022,

In [15]:
train_size = 1000
valid_size = 1000
batch_size = 128
epochs = 100

In [16]:
mpnn.fit([ip_data[:train_size],amat_data[:train_size]], y = op_data[:train_size], batch_size = batch_size, epochs = epochs, 
         callbacks = [lrate, stop_early], use_multiprocessing = False, initial_epoch = 0, verbose = 2, 
         validation_data = ([ip_data[train_size:train_size+valid_size],amat_data[train_size:train_size+valid_size]],op_data[train_size:train_size+valid_size]) )

Epoch 1/100
Learning rate:  0.001
8/8 - 18s - loss: -5.3202e-01 - mae: 0.5876 - log_mse: -7.4776e-01 - val_loss: -6.5605e-01 - val_mae: 0.5206 - val_log_mse: -9.8700e-01
Epoch 2/100
Learning rate:  0.001
8/8 - 11s - loss: -9.2450e-01 - mae: 0.3967 - log_mse: -1.5102e+00 - val_loss: -1.1033e+00 - val_mae: 0.3316 - val_log_mse: -2.0044e+00
Epoch 3/100
Learning rate:  0.001
8/8 - 11s - loss: -1.1592e+00 - mae: 0.3135 - log_mse: -2.0869e+00 - val_loss: -1.1502e+00 - val_mae: 0.3189 - val_log_mse: -2.0306e+00
Epoch 4/100
Learning rate:  0.001
8/8 - 11s - loss: -1.3361e+00 - mae: 0.2626 - log_mse: -2.2675e+00 - val_loss: -1.2060e+00 - val_mae: 0.3013 - val_log_mse: -2.0726e+00
Epoch 5/100
Learning rate:  0.001
8/8 - 11s - loss: -1.4214e+00 - mae: 0.2414 - log_mse: -2.3748e+00 - val_loss: -1.3062e+00 - val_mae: 0.2737 - val_log_mse: -2.2504e+00
Epoch 6/100
Learning rate:  0.001
8/8 - 12s - loss: -1.5094e+00 - mae: 0.2209 - log_mse: -2.5439e+00 - val_loss: -1.4213e+00 - val_mae: 0.2455 - val_l

KeyboardInterrupt: 